# yaml tutorial

이전에 argparse_tutorial을 진행하면서 어떻게 인자들을 파싱하는지를 배웠습니다.  
하지만 이렇게 입력하는 과정도 어쩐지 귀찮다는 생각이 듭니다.  
이 과정을 자동화하기 위해서 설정 파일을 만들어 놓고 이를 불러오는 식으로 사용하게 됩니다.  
보통 설정파일의 형식은 yaml 또는 json을 많이 사용합니다. yaml의 경우 하이퍼파라미터, json의 경우 데이터의 라벨을 주로 기록합니다.  
예를 들면 사용하는 형식은 다음과 같습니다. 

python run_cnn.py --model_path 'C:/Users/Aiffel/cnn/configs/cnn.yaml'

## config 파일 생성  
그럼 이제 직접 yaml 파일을 만들어 보고 불러와보겠습니다.  
1) Practice 폴더 내에 configs 폴더를 만드세요.   
2) configs 폴더 내에 cnn.yaml 파일을 만드세요.  
3) argparse에서 사용했던 하이퍼파라미터 정보들을 그대로 기록하고 저장하세요.  
4) yaml 파일 내에서는 키: 값 의 쌍으로 값을 저장합니다.  
ex)  
leaning_rate: 0.001

## yaml 파일 로드

In [1]:
import yaml


In [3]:
# with 구문으로 파일을 불러옵니다.

with open('configs/cnn.yaml') as f:
    config = yaml.load(f,Loader=yaml.FullLoader)
    print(type(config))

def print_config(**kwargs):
    for key, value in kwargs.items():
        print(key,value)

print_config(**config)


<class 'dict'>
batch_size 16
learning_rate 0.001
epochs 3
kernel_size 2
stride 2


# Quiz (Hard)   
이제 argparse와 yaml을 합쳐서 편하게 정보를 불러오는 실행파일을 만들어봅시다.  

1) run_cnn2 라는 파이썬 파일을 만드세요.  
2) argparse를 이용해 config_path라는 argument를 추가해줍니다. 인자의 타입은 스트링입니다.  
3) 위에서 구현한 yaml 파일을 로드하는 코드에서 config_path를 활용하여 하이퍼파라미터 정보를 불러오세요
4) 모든 하이퍼파라미터들을 출력해서 정상적으로 불러왔는지 확인하세요.  

In [10]:
%%writefile run_cnn2.py

'''
# Hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 5
kernel_size = 3
stride = 2
'''
import os 
import argparse

# parser 정의
parser = argparse.ArgumentParser(description='Argparse Tutorial')
# add_argument()를 통해 argument의 이름, 타입, 기본 값, 도움말을 정의할수 있다.
# parser.add_argument('-b','--batch_size', type=int, default=16, help="Score of korean")
# parser.add_argument('-l', '--learning_rate', type=float, default=0.1,help="Score of mathematcis")
# parser.add_argument('-e', '--epochs', type=int, default=1, help = "Score of english")
# parser.add_argument('-k', '--kernel_size', type=int, default=3, help = "Score of english")
# parser.add_argument('-s', '--stride', type=int, default=2, help = "Score of english")
parser.add_argument('-c', '--config_path', type=str, default='configs/cnn.yaml', help = "Score of english")



# add_argment()함수를 호출하면 parser인스턴스 내부에 해당 이름을 가지는 멤버 변수를 생성
# parse_arg()를 통해 프로그램 실행시 parser가 실행되도록 합니다.
args = parser.parse_args()

# subject_info = {'korean': args.n}
def run_cnn(args):
    

    

    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim
    import numpy as np

    from torch.utils.data import DataLoader, Dataset 
    from torchvision import datasets, transforms

    import yaml
    # with 구문으로 파일을 불러옵니다.

    with open(args.config_path) as f:
        config = yaml.load(f,Loader=yaml.FullLoader)
        print(type(config))

    # def print_config(**kwargs):
    #     for key, value in kwargs.items():
    #         if key == 'batch_size':
    #             batch_size = value
    #         if key == 'learning_rate':
    #             learning_rate = value    
    #         if key == 'epochs':
    #             epochs = value
    #         if key == 'kernel_size':
    #             kernel_size = value
    #         if key == 'stride':
    #             stride = value
            

    # Hyperparameters
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    epochs = config['epochs']
    kernel_size = config['kernel_size']
    stride = config['stride']

    # print_config(**config)

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    train_dataset = datasets.MNIST(root='./mnist_data/', train=True, download=True, transform=transforms.ToTensor())
    valid_dataset = datasets.MNIST(root='./mnist_data/', train=False, download=True, transform=transforms.ToTensor())
    test_dataset = datasets.MNIST(root='./mnist_data/', train=False, download=True, transform=transforms.ToTensor())
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    vaild_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

    shape = train_dataset[0][0].shape
    print(shape)
    C = shape[0]
    W = shape[1]
    H = shape[2]
    print(C, W, H)

    def train(epoch, model, loss_func, train_loader, optimizer):
        model.train()
        for batch_index, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = loss_func(y_pred, y)
            loss.backward()
            optimizer.step()
            if batch_index % 100 == 0:
                print(f'Train Epoch: {epoch+1} | Batch Status: {batch_index*len(x)}/{len(train_loader.dataset)} \
                ({100. * batch_index * batch_size / len(train_loader.dataset):.0f}% | Loss: {loss.item():.6f}')

    def test(model, loss_func, test_loader):
        model.eval()
        test_loss = 0
        correct_count = 0
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            test_loss += loss_func(y_pred, y).item()
            pred = y_pred.data.max(1, keepdim=True)[1]
            # torch.eq : Computes element-wise equality. return counts value
            correct_count += pred.eq(y.data.view_as(pred)).cpu().sum()
        
        test_loss /= len(test_loader.dataset)
        print(f'=======================\n Test set: Average loss: {test_loss:.4f}, Accuracy: {correct_count/len(test_loader.dataset):.3}')

    class CNN(nn.Module):
        def __init__(self, C, W, H, K, S): # 채널, 너비, 높이, 커널 사이즈, 스트라이드
            super(CNN, self).__init__()
            # nn.Module에는 이미 conv 레이어가 구현되어 있다. 
            # 배치정규화도 구현되어있고 다 구현되어있습니다. 
            self.conv1 = nn.Conv2d(C, 32, kernel_size=K, stride=S)
            self.bn1 = nn.BatchNorm2d(32)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=K, stride=S)
            self.bn2 = nn.BatchNorm2d(64)
            self.conv3 = nn.Conv2d(64, 128, kernel_size=K, stride=S)
            self.bn3 = nn.BatchNorm2d(128)
            
            def conv2d_size_out(size, kernel_size=K, stride=S):
                print((size - (kernel_size - 1) - 1) // stride + 1)
                return (size - (kernel_size - 1) - 1) // stride + 1
            
            convw = conv2d_size_out(W, K, S)
            convw = conv2d_size_out(convw, K, S)
            convw = conv2d_size_out(convw, K, S)
            
            self.linear_input_size = convw * convw * 128
            self.fc = nn.Linear(self.linear_input_size, 10)
            
        def forward(self, x):
            x = F.relu(self.bn1(self.conv1(x)))
            x = F.relu(self.bn2(self.conv2(x)))
            x = F.relu(self.bn3(self.conv3(x)))
            x = x.view(x.size(0), -1) # (batch_size, flatten_size)
            x = F.relu(self.fc(x))
            return F.log_softmax(x)

    cnn = CNN(C=C, W=W, H=H, K=kernel_size, S=stride) 
    cnn = cnn.to(device)
    ce_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        train(epoch, cnn, ce_loss, train_loader, optimizer)

    test(cnn, ce_loss, test_loader)
run_cnn(args)

Overwriting run_cnn2.py


In [11]:
!python3 run_cnn2.py

<class 'dict'>
torch.Size([1, 28, 28])
1 28 28
14
7
3
run_cnn2.py:146: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Train Epoch: 1 | Batch Status: 0/60000                 (0% | Loss: 2.449309
Train Epoch: 1 | Batch Status: 1600/60000                 (3% | Loss: 1.806441
Train Epoch: 1 | Batch Status: 3200/60000                 (5% | Loss: 1.225666
Train Epoch: 1 | Batch Status: 4800/60000                 (8% | Loss: 1.189337
Train Epoch: 1 | Batch Status: 6400/60000                 (11% | Loss: 1.456013
Train Epoch: 1 | Batch Status: 8000/60000                 (13% | Loss: 1.347386
Train Epoch: 1 | Batch Status: 9600/60000                 (16% | Loss: 1.831234
Train Epoch: 1 | Batch Status: 11200/60000                 (19% | Loss: 1.019140
Train Epoch: 1 | Batch Status: 12800/60000                 (21% | Loss: 1.041910
Train Epoch: 1 | Batch Status: 14400/60000                 (24%

## Json Tutorial  
json 파일을 작성하고 불러오는 과정은 yaml 파일과 굉장히 유사합니다.  

JSON 파일의 예제는 아래와 같습니다. 

{

   "이름": "홍길동",                          → 스트링

   "나이": 25,                                      → 숫자 (정수)

   "특기": ["농구", "도술"],              → list 표현 가능

    "가족관계": {"아버지": "홍판서", "어머니": "춘섬"},  → array 표현 가능

   "결혼 여부": true                          → Boolean 

}


## json 파일 생성  
마찬가지로 직접 json 파일을 만들어 보고 불러와보겠습니다.  
1) Practice 폴더 내에 configs 폴더를 만드세요.  
2) configs 폴더 내에 cnn.json 파일을 만드세요.  
3) argparse에서 사용했던 하이퍼파라미터 정보들을 그대로 기록하고 저장하세요.  
4) json 파일에서도 키: 값 의 쌍으로 값을 저장합니다.    
5) yaml과의 차이점은 키 값을 쌍따옴표""로 감싸야한다는 것과 {} 괄호 및 콤마, 를 사용하는 것입니다.  
